In [1]:
import pandas as pd
import talib as TA
import pandas_ta as ta
from backtesting import Strategy, Backtest

In [2]:
'''
This strategy takes into account RSI for entry signals and a exponential moving average for trend detection.
is intended to be used with a 15m timeframe, but can be used with any timeframe, just test it out.
The backtest is done with backtesting.py and is intended to throw the results from it, the results are going to be optimized.
'''

'\nThis strategy takes into account RSI for entry signals and a exponential moving average for trend detection.\nis intended to be used with a 15m timeframe, but can be used with any timeframe, just test it out.\nThe backtest is done with backtesting.py and is intended to throw the results from it, the results are going to be optimized.\n'

READ & CLEAN THE DATA SET

In [2]:
df = pd.read_csv('data_sets/EURUSD_Candlestick_15_M_BID_09.04.2004-04.04.2022.csv')
df = df[df['Volume']!=0]
df.drop(columns=['Unnamed: 0'], inplace=True)
df.reset_index(drop=True, inplace=True)
df

,Local time,Open,High,Low,Close,Volume
0,09.04.2004 00:00:00.000 GMT-0500,1.20722,1.20820,1.20689,1.20794,7953.20
1,09.04.2004 00:15:00.000 GMT-0500,1.20796,1.20855,1.20752,1.20824,7377.60
2,09.04.2004 00:30:00.000 GMT-0500,1.20817,1.20838,1.20767,1.20790,8624.70
3,09.04.2004 00:45:00.000 GMT-0500,1.20807,1.20834,1.20719,1.20820,6816.70
4,09.04.2004 01:00:00.000 GMT-0500,1.20780,1.20941,1.20773,1.20888,8339.30
...,...,...,...,...,...,...
449188,04.04.2022 17:45:00.000 GMT-0500,1.09742,1.09742,1.09717,1.09724,154.88
449189,04.04.2022 18:00:00.000 GMT-0500,1.09727,1.09735,1.09716,1.09722,148.93
449190,04.04.2022 18:15:00.000 GMT-0500,1.09722,1.09730,1.09710,1.09723,199.63
449191,04.04.2022 18:30:00.000 GMT-0500,1.09723,1.09761,1.09722,1.09745,184.76


CALCULATION OF ATR

In [3]:
df['ATR']= df.ta.atr()
df.tail()     

,Local time,Open,High,Low,Close,Volume,ATR
449188,04.04.2022 17:45:00.000 GMT-0500,1.09742,1.09742,1.09717,1.09724,154.88,0.000410
449189,04.04.2022 18:00:00.000 GMT-0500,1.09727,1.09735,1.09716,1.09722,148.93,0.000394
449190,04.04.2022 18:15:00.000 GMT-0500,1.09722,1.09730,1.09710,1.09723,199.63,0.000380
449191,04.04.2022 18:30:00.000 GMT-0500,1.09723,1.09761,1.09722,1.09745,184.76,0.000381
449192,04.04.2022 18:45:00.000 GMT-0500,1.09746,1.09757,1.09726,1.09749,254.23,0.000376


BACKTEST ATR RELATED SL&TP

In [7]:
class MyStrat(Strategy):
    initsize = 0.08
    mysize = initsize
    ema_window = 200
    rsi_window = 3
    
    def init(self):
        super().init()
        self.ema = self.I(TA.EMA, self.data.Close, self.ema_window)
        self.rsi = self.I(TA.RSI, self.data.Close, self.rsi_window)        

    def next(self):
        super().next()
        slatr = 1.2*self.data.ATR[-1]
        TPSLRatio = 1.5
        price = self.data.Close[-1] 
        
        if self.position:
            self.position.close()
            
        if self.data.Close >= self.ema and self.rsi <= 10:
            sl1 = price - slatr
            tp1 = price + slatr*TPSLRatio
            
            self.buy(sl=sl1, tp=tp1, size = self.mysize)
            
        if self.data.Close <= self.ema and self.rsi >= 90:
            sl2 = price + slatr
            tp2 = price - slatr*TPSLRatio
            
            self.sell(sl=sl2, tp=tp2, size = self.mysize)

bt = Backtest(df, MyStrat, cash=500, margin=1/50, commission=.00)
stat = bt.run()
print(stat)
plot = bt.plot(plot_volume=False)
print(plot)

/tmp/ipykernel_5485/391708153.py:33: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df, MyStrat, cash=500, margin=1/50, commission=.00)


Start                                     0.0
End                                  449192.0
Duration                             449192.0
Exposure Time [%]                    3.513634
Equity Final [$]                  3537.437525
Equity Peak [$]                   3560.728758
Return [%]                         607.487505
Buy & Hold Return [%]               -9.143666
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -5.786527
Avg. Drawdown [%]                   -0.330914
Max. Drawdown Duration                16529.0
Avg. Drawdown Duration              469.53163
# Trades                              10115.0
Win Rate [%]                        55.996045
Best Trade [%]                        0.47816
Worst Trade [%]                     -0.433835
Avg. Trade [%]                    

/usr/bin/sensible-browser: 8: /mnt/c/Program: not found
[7344:7381:0404/223504.770298:ERROR:bus.cc(397)] Failed to connect to the bus: Failed to connect to socket /var/run/dbus/system_bus_socket: No such file or directory
[7344:7381:0404/223504.770494:ERROR:bus.cc(397)] Failed to connect to the bus: Failed to connect to socket /var/run/dbus/system_bus_socket: No such file or directory
[7344:7380:0404/223504.775912:ERROR:bus.cc(397)] Failed to connect to the bus: Could not parse server address: Unknown address type (examples of valid types are "tcp" and on UNIX "unix")
[7344:7380:0404/223504.776030:ERROR:bus.cc(397)] Failed to connect to the bus: Could not parse server address: Unknown address type (examples of valid types are "tcp" and on UNIX "unix")
[7344:7380:0404/223504.788471:ERROR:bus.cc(397)] Failed to connect to the bus: Could not parse server address: Unknown address type (examples of valid types are "tcp" and on UNIX "unix")
[7344:7380:0404/223504.788549:ERROR:bus.cc(397)] Fa

OPTIMIZATION

In [9]:
stats = bt.optimize(
    ema_window = [50, 200],
    rsi_window = range(2, 4),
    maximize = 'Equity Final [$]',
)
print(stats)
print(stats._strategy)
bt.plot(plot_volume=False, plot_pl=False)

/usr/bin/sensible-browser: 8: /mnt/c/Program: not found


Backtest.optimize:   0%|          | 0/6 [00:00<?, ?it/s]

[1827:1870:0404/202138.039130:ERROR:bus.cc(397)] Failed to connect to the bus: Failed to connect to socket /var/run/dbus/system_bus_socket: No such file or directory
[1827:1870:0404/202138.039316:ERROR:bus.cc(397)] Failed to connect to the bus: Failed to connect to socket /var/run/dbus/system_bus_socket: No such file or directory
[1827:1869:0404/202138.044233:ERROR:bus.cc(397)] Failed to connect to the bus: Could not parse server address: Unknown address type (examples of valid types are "tcp" and on UNIX "unix")
[1827:1869:0404/202138.044314:ERROR:bus.cc(397)] Failed to connect to the bus: Could not parse server address: Unknown address type (examples of valid types are "tcp" and on UNIX "unix")
[1827:1869:0404/202138.071298:ERROR:bus.cc(397)] Failed to connect to the bus: Could not parse server address: Unknown address type (examples of valid types are "tcp" and on UNIX "unix")
[1827:1869:0404/202138.071356:ERROR:bus.cc(397)] Failed to connect to the bus: Could not parse server addre

Start                                     0.0
End                                   74762.0
Duration                              74762.0
Exposure Time [%]                   13.148215
Equity Final [$]                  5034.860812
Equity Peak [$]                   5509.687085
Return [%]                         906.972162
Buy & Hold Return [%]               -2.978043
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -20.370547
Avg. Drawdown [%]                   -1.094701
Max. Drawdown Duration                 4344.0
Avg. Drawdown Duration             179.493261
# Trades                               6467.0
Win Rate [%]                        52.605536
Best Trade [%]                        0.47816
Worst Trade [%]                     -0.215474
Avg. Trade [%]                    

Row(id='2396', ...)

/usr/bin/sensible-browser: 8: /mnt/c/Program: not found
[2201:2238:0404/202203.415737:ERROR:bus.cc(397)] Failed to connect to the bus: Failed to connect to socket /var/run/dbus/system_bus_socket: No such file or directory
[2201:2238:0404/202203.415976:ERROR:bus.cc(397)] Failed to connect to the bus: Failed to connect to socket /var/run/dbus/system_bus_socket: No such file or directory
[2201:2237:0404/202203.421480:ERROR:bus.cc(397)] Failed to connect to the bus: Could not parse server address: Unknown address type (examples of valid types are "tcp" and on UNIX "unix")
[2201:2237:0404/202203.421550:ERROR:bus.cc(397)] Failed to connect to the bus: Could not parse server address: Unknown address type (examples of valid types are "tcp" and on UNIX "unix")
[2201:2237:0404/202203.435965:ERROR:bus.cc(397)] Failed to connect to the bus: Could not parse server address: Unknown address type (examples of valid types are "tcp" and on UNIX "unix")
[2201:2237:0404/202203.436046:ERROR:bus.cc(397)] Fa